## Description
Minh họa các bước làm sạch tập dữ liệu GeneratedLabelledFlows từ CICIDS2017 dataset.

### Change the encoding to utf-8
Trước tiên ta cần thay đổi mã hóa của tệp CSV có tên Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv từ latin1 sang utf-8.
Dưới đây là đoạn mã Python thực hiện việc này:

In [15]:
import os
import codecs
import pandas as pd

"""
    Hàm này đọc một tệp với mã hóa đã cho và ghi lại nó với mã hóa utf-8.

    Tham số:
    - filename (str): đường dẫn đến tệp cần chuyển đổi
    - encoding (str): mã hóa ban đầu của tệp (mặc định là 'latin1')
    - blocksize (int): kích thước của khối dữ liệu đọc và ghi mỗi lần (mặc định là 1MB)
"""

def _to_utf8(filename: str, encoding="latin1", blocksize=1048576):
    tmpfilename = filename + ".tmp"
    with codecs.open(filename, "r", encoding) as source:
        with codecs.open(tmpfilename, "w", "utf-8") as target:
            while True:
                contents = source.read(blocksize)
                if not contents:
                    break
                target.write(contents)
                
    # Nếu tệp gốc tồn tại, xóa nó trước khi thay thế
    if os.path.exists(filename):
        os.remove(filename)

    # replace the original file
    os.rename(tmpfilename, filename)


Tiếp đến gọi hàm _to_utf8 để thay đổi mã hóa cho tệp cần xử lý:

In [16]:
file_name = os.path.join("GeneratedLabelledFlows/TrafficLabelling", "Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv")
_to_utf8(file_name)


### Removing rows with only NaN values
Tiếp đến chúng ta sẽ thực hiện đọc tệp Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv, kiểm tra và loại bỏ các dòng chứa giá trị NaN

In [23]:
import pandas as pd

# Đọc tệp CSV và loại bỏ các dòng chứa toàn bộ NaN
df = pd.read_csv(file_name, skipinitialspace=True, on_bad_lines='skip')

# Hiển thị số lượng các dòng chứa toàn bộ NaN
print("Removing {} rows that contains only NaN values...".format(df[df.isna().all(axis=1)].shape[0]))

# Loại bỏ các dòng chứa toàn bộ NaN
df = df[~df.isna().all(axis=1)]

# Kiểm tra kết quả
print(f"Số dòng còn lại: {df.shape[0]}")



C:\Users\vegap\AppData\Local\Temp\ipykernel_9400\3522157122.py:4: DtypeWarning: Columns (0,1,3,6,84) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_name, skipinitialspace=True, on_bad_lines='skip')


Removing 288602 rows that contains only NaN values...
Số dòng còn lại: 170366


### Change the unrecognized character in the class label
Tiếp đến ta sẽ thực hiện thay thế ký tự không nhận dạng trong label của lớp. Ở đây ký tự không nhận dạng **\x96** sẽ được thay thế bằng dấu gạch ngang thông thường **(-)**.

In [25]:
import pandas as pd

def _renaming_class_label(df: pd.DataFrame):
    labels = {
        "Web Attack \x96 Brute Force": "Web Attack-Brute Force",
        "Web Attack \x96 XSS": "Web Attack-XSS",
        "Web Attack \x96 Sql Injection": "Web Attack-Sql Injection"
    }

    # Thay thế nhãn không hợp lệ bằng nhãn hợp lệ
    for old_label, new_label in labels.items():
        df['Label'] = df['Label'].replace(old_label, new_label)

# Gọi hàm để thay đổi nhãn
_renaming_class_label(df)

# Kiểm tra kết quả
print(df['Label'].unique())  # Hiển thị các nhãn lớp đã được thay đổi


['BENIGN' 'Web Attack-Brute Force' 'Web Attack-XSS'
 'Web Attack-Sql Injection']


### Saving the dataset
Lưu trữ và kết hợp các tệp CSV từ thư mục TrafficLabelling, sau đó lưu kết quả vào một tệp CSV mới

In [9]:
import pandas as pd
import os

# Đường dẫn thư mục chứa các tệp dữ liệu
DIR_PATH = "GeneratedLabelledFlows/TrafficLabelling"

# Danh sách các tệp cần kết hợp
FILE_NAMES = [
    "Monday-WorkingHours.pcap_ISCX.csv",
    "Tuesday-WorkingHours.pcap_ISCX.csv",
    "Wednesday-workingHours.pcap_ISCX.csv",
    "Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv",
    "Thursday-WorkingHours-Afternoon-Infilteration.pcap_ISCX.csv",
    "Friday-WorkingHours-Morning.pcap_ISCX.csv",
    "Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX.csv",
    "Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv"
]

# Đọc tất cả các tệp CSV vào các DataFrame
df_list = [pd.read_csv(os.path.join(DIR_PATH, f), skipinitialspace=True) for f in FILE_NAMES]

# Kết hợp tất cả các DataFrame lại với nhau
df = pd.concat(df_list, ignore_index=True)

# Hiển thị số lượng các lớp trong cột 'Label'
print(df.Label.value_counts())

# Lưu tệp kết hợp vào tệp CSV mới
output_file = os.path.join(DIR_PATH, "TrafficLabelling.csv")
df.to_csv(output_file, index=False)

C:\Users\vegap\AppData\Local\Temp\ipykernel_15992\4153347537.py:20: DtypeWarning: Columns (0,1,3,6,84) have mixed types. Specify dtype option on import or set low_memory=False.
  df_list = [pd.read_csv(os.path.join(DIR_PATH, f), skipinitialspace=True) for f in FILE_NAMES]


Label
BENIGN                        2273097
DoS Hulk                       231073
PortScan                       158930
DDoS                           128027
DoS GoldenEye                   10293
FTP-Patator                      7938
SSH-Patator                      5897
DoS slowloris                    5796
DoS Slowhttptest                 5499
Bot                              1966
Web Attack  Brute Force         1507
Web Attack  XSS                  652
Infiltration                       36
Web Attack  Sql Injection         21
Heartbleed                         11
Name: count, dtype: int64
